In [0]:
%run ../../_utils

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached Unidecode-1.3.8-py3-none-any.whl (235 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import col, sum, count


# Camada GOLD

Na camada gold, as limpezas e ajustes já foram feitos, então essa camada é responsável por aplicar regras de negócio, agregações e junções de dados que convirjam para analises.

Essa é uma tabela sumarizada analítica.

O objetivo dela é informar o total de venda bruta por cada vendedor (aqui temos cidade)

In [0]:
tb_name = "gold.olist_total_orders_profit_by_seller_city"
dataset_location = "olist_total_orders_profitprofi_dataset"
target_location = f"dbfs:/FileStore/delta/gold/brazilian_ecommerce/{dataset_location}"

## 1 - Data ingestion


In [0]:
df_order_items = spark.read.table("silver.olist_order_items") # temos o seller_id
df_sellers = spark.read.table("silver.olist_sellers")


## 2 - preparation

In [0]:
df = df_order_items.join(df_sellers, on=['seller_id'], how="inner")

In [0]:
df = df.groupBy("seller_city").agg(
    count("order_id").alias("total_orders_by_seller_city"),
    sum("price").alias("total_items_sell_profit"), # total de arrecadação bruta por produto vendido
    sum("freight_value").alias("total_freight_profit") # total de arrecadação bruta por fretes
).withColumn("total_profit", col("total_items_sell_profit") + col("total_freight_profit")) # total arrecadação bruta

In [0]:
display(df.take(10))

seller_city,total_orders_by_seller_city,total_items_sell_profit,total_freight_profit,total_profit
igrejinha,3,314.96000000000004,79.25999999999999,394.22
brusque,393,47626.850000000115,10807.520000000008,58434.370000000126
buritama,15,2575.9000000000005,341.47,2917.370000000001
ipaussu,74,7268.0,1431.3200000000004,8699.32
carapicuiba,214,21859.62000000002,3615.1000000000004,25474.720000000023
garca,115,22979.630000000023,2612.8700000000003,25592.500000000022
sao joao de meriti,50,4970.830000000001,832.7500000000002,5803.580000000001
fernando prestes,3,86.6,38.669999999999995,125.26999999999998
araras,46,10790.85,912.2500000000002,11703.1
nova friburgo,90,62579.26000000003,2673.8599999999988,65253.12000000003



## Saving data

In [0]:
save_dataframe(df, format_mode="delta", table_name=tb_name, target_location=target_location)

[LOG] Saving gold.olist_total_orders_profit_by_seller_city delta on dbfs:/FileStore/delta/gold/brazilian_ecommerce/olist_total_orders_profitprofi_dataset... OK!



## create delta table

In [0]:
create_table(table_name=tb_name, target_location=target_location)

[LOG] Creating delta table gold.olist_total_orders_profit_by_seller_city on dbfs:/FileStore/delta/gold/brazilian_ecommerce/olist_total_orders_profitprofi_dataset... OK!


In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql

select * from gold.olist_total_orders_profit_by_seller_city limit 10

seller_city,total_orders_by_seller_city,total_items_sell_profit,total_freight_profit,total_profit
igrejinha,3,314.96000000000004,79.25999999999999,394.22
brusque,393,47626.850000000115,10807.520000000008,58434.370000000126
buritama,15,2575.9000000000005,341.47,2917.370000000001
ipaussu,74,7268.0,1431.3200000000004,8699.32
carapicuiba,214,21859.62000000002,3615.1000000000004,25474.720000000023
garca,115,22979.630000000023,2612.8700000000003,25592.500000000022
sao joao de meriti,50,4970.830000000001,832.7500000000002,5803.580000000001
fernando prestes,3,86.6,38.669999999999995,125.26999999999998
araras,46,10790.85,912.2500000000002,11703.1
nova friburgo,90,62579.26000000003,2673.8599999999988,65253.12000000003
